<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# Redes Neuronales Convulucionales (CNN) multivariadas para series de tiempo
## *Modelos no lineales para pronósitico*  - Pedro Martinez

---

# CNN 1D multivariadas para pronóstico del clima

Hoy en día, Google y OpenWeather usan variantes de CNN en sus pipelines de nowcasting meteorológico ya que son capaces de detectar patrones temporales locales en series de tiempo, como picos de temperatura o tendencias estacionales.

Usaremos datos reales de la API [Open-Meteo](https://open-meteo.com/) para **predecir la temperatura de las próximas 48 h**, empleando las variables:

| Variable | Descripción | Unidad | Fuente |
|-----------|--------------|---------|---------|
| **Temperatura** | Temperatura del aire a 2 metros | °C (grados Celsius) | `temperature_2m` |
| **Humedad relativa** | Porcentaje de saturación de vapor de agua | % | `relative_humidity_2m` |
| **Nubosidad** | Cobertura nubosa promedio | % | `cloud_cover` |
| **Velocidad del viento** | Velocidad del viento a 10 metros | m/s | `wind_speed_10m` |

---

Una CNN aplicada a series de tiempo multivariables procesa una secuencia de observaciones:

$$
X_t =
\begin{bmatrix}
x_{t,1} & x_{t,2} & \dots & x_{t,n}
\end{bmatrix}
\in \mathbb{R}^{n}
$$

donde $n$ es el número de variables de entrada (en este caso, 4).

Si utilizamos una **ventana temporal de tamaño $T$**, la entrada al modelo es:

$$
\mathbf{X} =
\begin{bmatrix}
X_{t-T+1} \\
X_{t-T+2} \\
\vdots \\
X_t
\end{bmatrix}
\in \mathbb{R}^{T \times n}
$$

Cada capa convolucional aplica un conjunto de filtros (*kernels*) que aprenden patrones espacio-temporales en la ventana:

$$
h_j^{(l)}(t) = f \!\left(
\sum_{i=1}^{n}
\sum_{k=0}^{K-1}
w_{ij}^{(l)}(k) \, x_i(t - k)
+ b_j^{(l)}
\right)
$$

donde:

- $w_{ij}^{(l)}(k)$ son los pesos del filtro $j$ en la posición $k$ de la capa $l$.  
- $b_j^{(l)}$ es el sesgo asociado al filtro $j$.  
- $f(\cdot)$ es una función de activación no lineal (típicamente **ReLU**).  
- $K$ es el tamaño del kernel (por ejemplo, 3 horas).

El resultado de esta capa convolucional se **aplasta (*flatten*)** y se conecta a una capa **densa (fully connected)** para generar la predicción final.

---

## Interpretación

- La CNN aprende **patrones locales** (por ejemplo, cómo la humedad y la nubosidad afectan la temperatura en las últimas horas).  
- Al combinar varias variables de entrada, la red actúa como un **modelo multivariable no lineal**.  
- El entrenamiento minimiza una función de pérdida (MSE):


In [154]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import requests
import pandas as pd
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers, models
import plotly.graph_objects as go
from datetime import datetime, timedelta
import pytz

In [155]:
CITY_TZ = "America/Mexico_City"
LAT, LON = 20.31, -103.18  # Chapala
PAST_DAYS = 15

tz = pytz.timezone(CITY_TZ)
url = (
    "https://api.open-meteo.com/v1/forecast"
    f"?latitude={LAT}&longitude={LON}"
    "&hourly=temperature_2m,relative_humidity_2m,cloud_cover,wind_speed_10m"
    f"&past_days={PAST_DAYS}"
    f"&timezone={CITY_TZ}"
)
res = requests.get(url).json()

now_local = datetime.now(tz)
# Dejamos fuera la hora en curso para no usar datos parciales
cutoff = now_local - timedelta(hours=1)

# Construir TODO (histórico + futuro) a partir de la misma respuesta `res`
df_full = pd.DataFrame({
    "time": pd.to_datetime(res["hourly"]["time"]),
    "temp": res["hourly"]["temperature_2m"],
    "humidity": res["hourly"]["relative_humidity_2m"],
    "clouds": res["hourly"]["cloud_cover"],
    "wind": res["hourly"]["wind_speed_10m"],
}).set_index("time")

df_full.index = df_full.index.tz_localize(tz) if df_full.index.tz is None else df_full.index.tz_convert(tz)

# Reutiliza tu 'cutoff' (última hora cerrada)
df_hist  = df_full[df_full.index <= cutoff].copy()
df_fut48 = df_full[(df_full.index > cutoff)].iloc[:48].copy()  # primeras 48 horas futuras

print("Histórico hasta:", df_hist.index[-1])
print("Futuro 48h desde:", df_fut48.index[0], "hasta:", df_fut48.index[-1])


df = pd.DataFrame({
    "time": pd.to_datetime(res["hourly"]["time"]),
    "temp": res["hourly"]["temperature_2m"],
    "humidity": res["hourly"]["relative_humidity_2m"],
    "clouds": res["hourly"]["cloud_cover"],
    "wind": res["hourly"]["wind_speed_10m"],
}).set_index("time")

df.index = df.index.tz_localize(tz)
now_local = datetime.now(tz)
cutoff = now_local - timedelta(hours=1)
df = df[df.index <= cutoff]

px.line(df, y=["temp","humidity","clouds","wind"],
        title="Variables meteorológicas", template="plotly_white").show()


Histórico hasta: 2025-10-31 18:00:00-06:00
Futuro 48h desde: 2025-10-31 19:00:00-06:00 hasta: 2025-11-02 18:00:00-06:00


In [156]:
fig = px.line(df, y=["temp","humidity","clouds","wind"],
              title="Variables meteorológicas",
              template="plotly_white")
fig.show()

En equipos de 2 personas, realizar las siguientes partes:

Seleccionar variable objetivo

Crear ventanas temporales


*   Se sugiere utilizar 12 o 24 valores anteriores


In [157]:
X_FEATURES = ["temp", "humidity", "clouds"]
TARGET_COL = "wind"
WINDOW = 24  # puedes probar 12 o 24

def create_sequences(X_scaled, y_scaled, window):
    X, y = [], []
    X_vals, y_vals = X_scaled.values, y_scaled.values
    for i in range(len(X_scaled) - window):
        X.append(X_vals[i:i+window, :])
        y.append(y_vals[i+window])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

Escalar los datos

In [158]:
# Escaladores separados
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_scaled = pd.DataFrame(
    x_scaler.fit_transform(df[X_FEATURES]),
    columns=X_FEATURES, index=df.index
)
y_scaled = pd.Series(
    y_scaler.fit_transform(df[[TARGET_COL]]).ravel(),
    index=df.index, name=TARGET_COL
)

X, y = create_sequences(X_scaled, y_scaled, WINDOW)

Dividir en entrenamiento y prueba

In [159]:
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(X_train.shape, X_test.shape)

(284, 24, 3) (71, 24, 3)


Construir la CNN en Keras
- Recordando las recomendaciones de la clase pasada
- Cambiar número de filtros (16, 32, 64)
- Cambiar tamaño del kernel (2, 3, 5)
- Cambiar función de activación (relu, tanh)
- Cambiar batch_size o epochs

In [160]:
FILTERS = [
    (64, "gelu"),
    (32, "gelu")
]
KERNEL = 3

def build_cnn(window, n_features, filters, kernel_size):
    model = models.Sequential([layers.Input(shape=(window, n_features))])
    
    for f, act in filters:
        model.add(layers.Conv1D(
            filters=f,
            kernel_size=kernel_size,
            activation=act,
            padding='causal'
        ))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="swish"))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

model = build_cnn(WINDOW, len(X_FEATURES), FILTERS, KERNEL)
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_27 (Conv1D)              │ (None, 24, 64)         │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_28 (Conv1D)              │ (None, 24, 32)         │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │        49,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,097 (219.13 KB)

 Trainable params: 56,097 (219.13 KB)

 Non-trainable params: 0 (0.00 B)

Entrenar el modelo utilizando una semilla para reproducibilidad

In [161]:
SEED = 42
EPOCHS = 30
BATCH = 32
np.random.seed(SEED)
tf.random.set_seed(SEED)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH,
    verbose=1
)

Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0296 - val_loss: 0.0795
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0207 - val_loss: 0.0451
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0156 - val_loss: 0.0455
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0157 - val_loss: 0.0475
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0150 - val_loss: 0.0388
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0133 - val_loss: 0.0377
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0130 - val_loss: 0.0372
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0132 - val_loss: 0.0374
Epoch 9/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0127 - val_loss: 0.0377
Epoch 10/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0120 - val_loss: 0.0370
Epoch 11/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0111 - val_loss: 0.0363
Epoch 12/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0106 - val_loss: 0.0363
Ep

Evaluar y visualizar utilizando plotly

In [162]:
y_pred_test_scaled = model.predict(X_test).flatten()
y_test_inv = y_scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()
y_pred_inv = y_scaler.inverse_transform(y_pred_test_scaled.reshape(-1, 1)).ravel()

mae = np.mean(np.abs(y_test_inv - y_pred_inv))
rmse = np.sqrt(np.mean((y_test_inv - y_pred_inv) ** 2))
print(f"MAE: {mae:.3f} m/s | RMSE: {rmse:.3f} m/s")

fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test_inv, mode='lines', name='Real'))
fig.add_trace(go.Scatter(y=y_pred_inv, mode='lines', name='Predicción'))
fig.update_layout(title="Evaluación en conjunto de prueba (wind)",
                  template="plotly_white")
fig.show()

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
MAE: 3.206 m/s | RMSE: 4.760 m/s


In [163]:
def forecast_48h_with_future_exog(
    X_hist_scaled: pd.DataFrame,   # exógenas (temp/humidity/clouds) escaladas HISTÓRICAS
    X_future: pd.DataFrame,        # exógenas futuras en ESCALA ORIGINAL
    x_scaler: MinMaxScaler,        # scaler de X
    model,
    x_features,
    window: int,
    city_tz: str,
    y_scaler: MinMaxScaler         # scaler de y para desescalar
):
    # Escalar exógenas futuras con el MISMO scaler de X
    X_future_scaled = pd.DataFrame(
        x_scaler.transform(X_future[x_features]),
        columns=x_features,
        index=X_future.index
    )

    # Concatenar histórico + futuro para poder construir ventanas deslizantes reales
    X_all = pd.concat([X_hist_scaled, X_future_scaled], axis=0)

    # El primer índice de predicción es en la posición donde ya tengo 'window' horas previas completas
    start = len(X_hist_scaled) - window
    preds_scaled = []

    # Para 48 pasos: cada paso usa una ventana que avanza 1 hora e incluye exógenas futuras reales
    for step in range(48):
        left = start + step
        right = left + window
        X_win = X_all.iloc[left:right].values[np.newaxis, ...]  # (1, window, n_features)
        yhat_scaled = model.predict(X_win, verbose=0)[0, 0]
        preds_scaled.append(yhat_scaled)

    # Desescalar a m/s
    wind_future = y_scaler.inverse_transform(np.array(preds_scaled).reshape(-1, 1)).ravel()

    # Índice temporal = el de X_future (48 timestamps futuros)
    pred_df = pd.DataFrame({"pred_wind": wind_future}, index=X_future.index[:48])
    pred_df.index = pred_df.index.tz_convert(pytz.timezone(city_tz))
    return pred_df

# Ejecutar forecast con exógenas futuras
X_hist_scaled = X_scaled  # (de tu celda de escalado previa)
pred_df = forecast_48h_with_future_exog(
    X_hist_scaled=X_hist_scaled,
    X_future=df_fut48[X_FEATURES],
    x_scaler=x_scaler,
    model=model,
    x_features=X_FEATURES,
    window=WINDOW,
    city_tz=CITY_TZ,
    y_scaler=y_scaler
)

In [164]:
# Visualización final
fig = go.Figure()
hist = df_hist["wind"].iloc[-48:] if len(df_hist) >= 48 else df_hist["wind"]
fig.add_trace(go.Scatter(x=hist.index, y=hist.values, mode='lines', name='Últimas observadas (wind)'))
fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df["pred_wind"], mode='lines+markers', name='Pronóstico 48 h (wind) con exógenas reales'))
fig.update_layout(title="Pronóstico 48 h de viento (m/s) con exógenas futuras reales",
                  xaxis_title="Hora local", yaxis_title="m/s",
                  template="plotly_white")
fig.show()